## Visualizations

In [109]:
%useLatestDescriptors
%use dataframe
%use kandy

In [139]:
fun DataFrame.Companion.readMetricsJson(json: Path) = readJson(json.pathString).cast<Int>()

val resultDir = Path("results_perses/instances")
val metricsPath = Path("seru_result/metrics.json")
val instances = resultDir.toFile()
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .filter { !it.parent.endsWith("issue_2490_inlined") }
    .map { it.toPath() }


In [140]:
val instancesDfs = instances
    .map {
        it.listDirectoryEntries()
            .map { DataFrame.readMetricsJson(it / metricsPath) }
            .reduce { acc, dataFrame -> acc.concat(dataFrame) }
    }
    .reduce { acc, df -> acc.concat(df) }

In [141]:
val groupedInstanceDfs = instancesDfs
    .groupBy { InputDir.map { it.removePrefix("study/instances/") } }
    .sortBy { InputDir.map { it.substringBefore("/") } }

In [143]:
val meanDf = groupedInstanceDfs.mean()

meanDf
    .plot {
        bars {
            x(InputDir) {
                axis {
                    name = "Instance"
                }
            }
            y("TotalTimeMillis")
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zkG5h8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"TotalTimeMillis":[14115.4,1861.6,10794.6,4769.8,38301.2,21920.6,21996.0,13925.8,10475.4,1689.2,3178.4,2237.6,2187.8],
"InputDir":["error/issue_2246/v1","error/issue_2246/final","error/issue_2473/v1","error/issue_2473/final","error/issue_2209/v1","error/issue_2209/final","extra/issue_2_inlined/final","panic/issue_2584/v1","panic/issue_2584/v2","panic/issue_2584/final","semantic/issue_2218/v1","semantic/issue_2218/v2","semantic/issue_2218/final"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Instance"
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"InputDir",
"y":"TotalTimeMillis"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("zkG5h8");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 error/issue_2246/v1 
 
 
 
 
 
 
 
 
 error/issue_2246/final 
 
 
 
 
 
 
 
 
 error/issue_2473/v1 
 
 
 
 
 
 
 
 
 error/issue_2473/final 
 
 
 
 
 
 
 
 
 error/issue_2209/v1 
 
 
 
 
 
 
 
 
 error/issue_2209/final 
 
 
 
 
 
 
 
 
 extra/issue_2_inlined/final 
 
 
 
 
 
 
 
 
 panic/issue_2584/v1 
 
 
 
 
 
 
 
 
 panic/issue_2584/v2 
 
 
 
 
 
 
 
 
 panic/issue_2584/final 
 
 
 
 
 
 
 
 
 semantic/issue_2218/v1 
 
 
 
 
 
 
 
 
 semantic/issue_2218/v2 
 
 
 
 
 
 
 
 
 semantic/issue_2218/final 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 
 
 TotalTimeMillis 
 
 
 
 
 Instance